# data sampler

In [ ]:
#| default_exp data_sampler

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import os,pickle

## Setup

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

In [ ]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'
pkl_file = f'{pkl_dir}/processed/wikiseealso_data-meta_distilbert-base-uncased_rm_ramen-cat.pkl'

In [ ]:
with open(pkl_file, 'rb') as file: block = pickle.load(file)